In [1]:
import numpy as np
from scipy import signal
from keras.datasets import mnist
from PIL import Image

2023-12-12 22:13:54.991831: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### CNN Layer

In [2]:
class CNNLayer:
    def __init__(self, kernel_size, number_of_kernels, input_shape):
        assert len(input_shape) == 3, "Input shape must be 3D" 
        self.input_channels = input_shape[2]
        
        self.input_shape = input_shape
        self.kernel_size = kernel_size
        self.number_of_kernels = number_of_kernels
        self.kernels = np.random.randn(kernel_size, kernel_size, self.input_channels, number_of_kernels)
        
        # output shape
        self.output_shape = (input_shape[0] - kernel_size + 1, input_shape[1] - kernel_size + 1, number_of_kernels)
        self.biases = np.random.randn(*self.output_shape)
    
    def forward(self, input):
        assert input.shape == self.input_shape, "Input shape is not correct. " + "Expected: " + str(self.input_shape) + " Actual: " + str(input.shape)

        output = np.zeros(self.output_shape)
        for i in range(self.number_of_kernels):
            for j in range(self.input_channels):
                output[:, :, i] += signal.correlate2d(input[:, :, j], self.kernels[:, :, j, i], mode='valid')
        
        output += self.biases
        
        return output
    
    def backward(self, input, gradWRTMyOutput, learning_rate):
        assert gradWRTMyOutput.shape == self.output_shape, "Grad shape is not correct at CNN layer with kernel size: " + str(self.kernel_size) + " Expected: " + str(self.output_shape) + " Actual: " + str(gradWRTMyOutput.shape)
        assert input.shape == self.input_shape, "Input shape is not correct at CNN layer with kernel size: " + str(self.kernel_size) + " Expected: " + str(self.input_shape) + " Actual: " + str(input.shape)
        
        # Compute gradient with respect to kernels
        kernel_grads = np.zeros(self.kernels.shape)
        for i in range(self.number_of_kernels):
            for j in range(self.input_channels):
                kernel_grads[:, :, j, i] = signal.correlate2d(input[:, :, j], gradWRTMyOutput[:, :, i], mode='valid')
        
        # Compute gradient with respect to biases
        bias_grads = gradWRTMyOutput.copy()
        
        # Compute gradient with respect to input
        input_grads = np.zeros(input.shape)
        for j in range(self.input_channels):
            for i in range(self.number_of_kernels):
                input_grads[:, :, j] += signal.convolve2d(gradWRTMyOutput[:, :, i], self.kernels[:, :, j, i], mode='full')
        
        # Update kernels and biases
        self.kernels -= learning_rate * kernel_grads
        self.biases -= learning_rate * bias_grads
        
        return input_grads

#### Sigmoid Layer
Input: (x, y, z) where z is the number of channels

Output: (x', y', z')

Error_Grad_WRT_Input: Error_Grad_WRT_Output * Output_Grad_WRT_Input , where * is element-wise multiplication

In [3]:
class SigmoidLayer:
    def __init__(self, input_shape):
        # Input shape is 3D
        assert len(input_shape) == 3, "Input shape must be 3D"
        self.input_shape = input_shape
        self.output_shape = input_shape
    
    def forward(self, input):
        assert input.shape == self.input_shape, "Input shape is not correct." + " Expected: " + str(self.input_shape) + " Actual: " + str(input.shape)
        self.output = 1.0 / (1.0 + np.exp(-input))
        return self.output
    
    def sigmoid_derivative(self, x):
        return x * (1.0 - x)
    
    def backward(self, input, gradWRTMyOutput, learning_rate): # learning_rate is not used, but it is here to keep the same interface
        assert gradWRTMyOutput.shape == self.output_shape, "Grad shape is not correct at sigmoid layer." + " Expected: " + str(self.output_shape) + " Actual: " + str(gradWRTMyOutput.shape)
        assert input.shape == self.input_shape, "Input shape is not correct at sigmoid layer." + " Expected: " + str(self.input_shape) + " Actual: " + str(input.shape)
        # element-wise gradient
        gradMyOutputWRTMyInput = self.sigmoid_derivative(self.output)
        # element-wise multiplication
        return gradWRTMyOutput * gradMyOutputWRTMyInput

#### Average Pooling Layer
Input: (x, y, z) where z is the number of channels

Output: (x', y', z)

Error Gradient

$\frac{dE}{dX_{ijk}} = \sum_{m,n} \frac{dE}{dY_{mnk}} \times \frac{1}{\text{pool\_size}^2} \times \mathbb{\theta}_{i, j}$ where $\mathbb{\theta}_{i, j}$ is 1 if $X_{ijk}$ was included when calculating $Y_{mnk}$ and 0 otherwise.

In [4]:
class AveragePoolLayer:
    def __init__(self, input_shape, pool_size, stride):
        # Input shape is 3D
        assert len(input_shape) == 3, "Input shape must be 3D"
        # input shape must be divisible by pool size
        assert input_shape[0] % pool_size == 0, "Input shape must be divisible by pool size"
        self.input_shape = input_shape
        self.output_shape = (input_shape[0] // stride, input_shape[1] // stride, input_shape[2])
        self.pool_size = pool_size
        self.stride = stride
    
    def forward(self, input):
        assert input.shape == self.input_shape, "Input shape is not correct." + " Expected: " + str(self.input_shape) + " Actual: " + str(input.shape)
        self.output = np.zeros(self.output_shape)
        for i in range(self.output_shape[0]):
            for j in range(self.output_shape[1]):
                for k in range(self.output_shape[2]):
                    self.output[i, j, k] = np.mean(input[i*self.stride:i*self.stride+self.pool_size, j*self.stride:j*self.stride+self.pool_size, k])
        return self.output

    def backward(self, input, gradWRTMyOutput, learning_rate): # learning_rate is not used, but it is here to keep the same interface
        assert gradWRTMyOutput.shape == self.output_shape, "Grad shape is not correct at average pool layer." + " Expected: " + str(self.output_shape) + " Actual: " + str(gradWRTMyOutput.shape)
        assert input.shape == self.input_shape, "Input shape is not correct at average pool layer." + " Expected: " + str(self.input_shape) + " Actual: " + str(input.shape)
        
        # Compute gradient with respect to input
        avg_pool_n = self.pool_size * self.pool_size # number of elements involved in the average pool operation
        input_grads = np.zeros(input.shape)
        
        for channel in range(self.input_shape[2]):
            for i in range(self.output_shape[0]):
                for j in range(self.output_shape[1]):
                    # Only loop over the elements involved in the average pool operation
                    for x in range(i*self.stride, i*self.stride+self.pool_size):
                        for y in range(j*self.stride, j*self.stride+self.pool_size):
                            input_grads[x, y, channel] += gradWRTMyOutput[i, j, channel] / avg_pool_n
        
        return input_grads

#### Flatten Layer

In [5]:
class FlattenLayer():
    def __init__(self, input_shape):
        self.input_shape = input_shape
        self.output_shape = (np.prod(input_shape),)
    
    def forward(self, input):
        assert input.shape == self.input_shape, "Input shape is not correct"
        self.output = input.flatten()
        return self.output
    
    def backward(self, input, gradWRTMyOutput, learning_rate): # learning_rate is not used, but it is here to keep the same interface
        assert gradWRTMyOutput.shape == self.output_shape, "Grad shape is not correct at flatten layer." + " Expected: " + str(self.output_shape) + " Actual: " + str(gradWRTMyOutput.shape)
        assert input.shape == self.input_shape, "Input shape is not correct at flatten layer." + " Expected: " + str(self.input_shape) + " Actual: " + str(input.shape)
        return gradWRTMyOutput.reshape(self.input_shape)

#### Softmax Layer
Softmax Derivative: $\frac{dS_i}{dX_j} = S_i(1 - S_i)$ if $i = j$ and $-S_iS_j$ otherwise

In matrix form, obtain $C = softmax(x) \times softmax(x)^T$ and $D = diag(softmax(x))$ where $D$ is a diagonal matrix with the softmax values on the diagonal and 0 elsewhere.

$K = D - C$
Then, $\frac{dS}{dX} = K \times \frac{dE}{dO}$ where $O$ is the output of the softmax layer and $E$ is the error.

In [6]:
class SoftmaxLayer:
    def __init__(self, input_shape):
        # Input shape is 1D
        assert len(input_shape) == 1, "Input shape must be 1D"
        self.input_shape = input_shape
        self.output_shape = input_shape
    
    def softmax(self, x):
        return np.exp(x) / np.sum(np.exp(x))
    
    def forward(self, input):
        assert input.shape == self.input_shape, "Input shape is not correct"
        self.output = np.exp(input) / np.sum(np.exp(input))
        return self.output
    
    # def backward(self, input, gradWRTMyOutput, learning_rate): # learning_rate is not used, but it is here to keep the same interface
    #     assert gradWRTMyOutput.shape == self.output_shape, "Grad shape is not correct at softmax layer." + " Expected: " + str(self.output_shape) + " Actual: " + str(gradWRTMyOutput.shape)
    #     assert input.shape == self.input_shape, "Input shape is not correct at softmax layer." + " Expected: " + str(self.input_shape) + " Actual: " + str(input.shape)
        
    #     # Compute gradient with respect to input
    #     input_grads = np.zeros(input.shape)
    #     inp_softmax = self.softmax(input)
    #     softmax_matrix = np.reshape(inp_softmax, (inp_softmax.shape[0], 1)) @ np.reshape(inp_softmax, (1, inp_softmax.shape[0]))
    #     softmax_matrix = -1 * softmax_matrix
    #     np.fill_diagonal(softmax_matrix, inp_softmax * (1 - inp_softmax))
    #     input_grads = softmax_matrix @ gradWRTMyOutput
        
    #     return input_grads
    def backward(self, input, gradWRTMyOutput, learning_rate):
        assert gradWRTMyOutput.shape == self.output_shape
        assert input.shape == self.input_shape

        # Compute the softmax for the input
        inp_softmax = self.softmax(input)

        # Create an empty Jacobian matrix
        jacobian_matrix = np.zeros((len(inp_softmax), len(inp_softmax)))

        # Fill in the Jacobian matrix
        for i in range(len(inp_softmax)):
            for j in range(len(inp_softmax)):
                if i == j:
                    jacobian_matrix[i][j] = inp_softmax[i] * (1 - inp_softmax[i])
                else:
                    jacobian_matrix[i][j] = -inp_softmax[i] * inp_softmax[j]

        # Compute the gradient
        input_grads = np.dot(jacobian_matrix, gradWRTMyOutput)

        return input_grads

#### Cross Entropy Loss Layer

$\text{Loss} = -\frac{1}{N} \sum_{i=1}^{N} \left[ y_i \cdot \log(p_i) + (1 - y_i) \cdot \log(1 - p_i) \right]$


$\frac{\partial \text{Loss}}{\partial p_i} = -\frac{1}{N} \left( \frac{y_i}{p_i} - \frac{1 - y_i}{1 - p_i} \right)$

In [7]:
class CrossEntropyLoss:
    def __init__(self, input_shape):
        # Input shape is 1D, typically the output of a softmax layer for two classes
        assert len(input_shape) == 1, "Input shape must be 1D"
        self.input_shape = input_shape
        self.output_shape = (1,)

    def forward(self, input, target):
        # Ensuring input and target shapes are correct
        assert input.shape == self.input_shape, "Input shape is not correct. Expected: " + str(self.input_shape) + " Actual: " + str(input.shape)
        assert target.shape == self.input_shape, "Target shape is not correct. Expected: " + str(self.input_shape) + " Actual: " + str(target.shape)

        # Calculating cross entropy loss
        # Adding a small value to prevent log(0)
        eps = 1e-15
        self.output = -np.mean(target * np.log(input + eps) + (1 - target) * np.log(1 - input + eps))
        return self.output

    def backward(self, input, target, learning_rate): # learning_rate is not used, but it is here to keep the same interface
        # Ensuring input and target shapes are correct
        assert input.shape == self.input_shape, "Input shape is not correct at loss layer. Expected: " + str(self.input_shape) + " Actual: " + str(input.shape)
        assert target.shape == self.input_shape, "Target shape is not correct at loss layer. Expected: " + str(self.input_shape) + " Actual: " + str(target.shape)

        # Calculating gradient
        # Adding a small value to prevent division by zero
        eps = 1e-15
        return -(target / (input + eps) - (1 - target) / (1 - input + eps)) / input.size

#### Reshape Layer (Helper Layer)

In [8]:
class ReshapeLayer:
    def __init__(self, input_shape, output_shape):
        self.input_shape = input_shape
        self.output_shape = output_shape
    
    def forward(self, input):
        assert input.shape == self.input_shape, "Input shape is not correct. " + "Expected: " + str(self.input_shape) + " Actual: " + str(input.shape)
        self.output = input.reshape(self.output_shape)
        return self.output
    
    def backward(self, input, gradWRTMyOutput, learning_rate): # learning_rate is not used, but it is here to keep the same interface
        assert gradWRTMyOutput.shape == self.output_shape, "Grad shape is not correct at reshape layer." + " Expected: " + str(self.output_shape) + " Actual: " + str(gradWRTMyOutput.shape)
        assert input.shape == self.input_shape, "Input shape is not correct at reshape layer." + " Expected: " + str(self.input_shape) + " Actual: " + str(input.shape)
        return gradWRTMyOutput.reshape(self.input_shape)

In [9]:
def isCorrect(prediction, target):
    return np.argmax(prediction) == np.argmax(target)

In [10]:
# Load the MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# Display first image
Image.fromarray(x_train[0])

In [11]:
# Normalize the data
x_train = x_train / 255
x_test = x_test / 255

# Reshape the data
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1) # (height, width, channel)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1) # (height, width, channel)

# One-hot encode the labels
y_train = np.eye(10)[y_train]
y_test = np.eye(10)[y_test]

# Shuffle the data
np.random.seed(0)
indices = np.arange(x_train.shape[0])
np.random.shuffle(indices)
x_train = x_train[indices]
y_train = y_train[indices]

In [12]:
# Create the model
model = [
    CNNLayer(kernel_size=3, number_of_kernels=2, input_shape=(28, 28, 1)),
    SigmoidLayer(input_shape=(26, 26, 2)),
    AveragePoolLayer(input_shape=(26, 26, 2), pool_size=2, stride=2),
    FlattenLayer(input_shape=(13, 13, 2)),
    ReshapeLayer(input_shape=(338,), output_shape=(1,1,338)),
    CNNLayer(kernel_size=1, number_of_kernels=10, input_shape=(1,1,338)),
    ReshapeLayer(input_shape=(1,1,10), output_shape=(10,)),
    SoftmaxLayer(input_shape=(10,))
]

loss_layer = CrossEntropyLoss(input_shape=(10,))

# Forward pass
x = x_train[1]
y = y_train[1]
x = model[0].forward(x)
x = model[1].forward(x)
x = model[2].forward(x)
x = model[3].forward(x)
x = model[4].forward(x)
x = model[5].forward(x)
x = model[6].forward(x)
x = model[7].forward(x)
loss = loss_layer.forward(x, y)
print("Loss: ", loss)

Loss:  1.9044925042331073


In [13]:
# Train 
epochs = 5
learning_rate = 0.1
gamma = 0.5
interval = 1
for epoch in range(epochs):
    if epoch > 0 and epoch % interval == 0:
        learning_rate = learning_rate * gamma
    train_correct = 0
    for i in range(len(x_train)):
        x = x_train[i]
        y = y_train[i]
        
        # Forward pass
        layer0 = model[0].forward(x) # CNN with kernel size 3
        layer1 = model[1].forward(layer0) # Sigmoid
        layer2 = model[2].forward(layer1) # Average pool
        layer3 = model[3].forward(layer2) # Flatten
        layer4 = model[4].forward(layer3) # Reshape
        layer5 = model[5].forward(layer4) # CNN with kernel size 1
        layer6 = model[6].forward(layer5) # Reshape
        layer7 = model[7].forward(layer6) # Softmax
        
        # loss
        loss = loss_layer.forward(layer7, y) # Cross entropy
        # accuracy
        train_correct += isCorrect(layer7, y)
        
        # Grad
        grad = loss_layer.backward(layer7, y, learning_rate) # Cross entropy
        grad = model[7].backward(layer6, grad, learning_rate) # Softmax
        grad = model[6].backward(layer5, grad, learning_rate) # Reshape
        grad = model[5].backward(layer4, grad, learning_rate) # CNN with kernel size 1
        grad = model[4].backward(layer3, grad, learning_rate) # Reshape
        grad = model[3].backward(layer2, grad, learning_rate) # Flatten
        grad = model[2].backward(layer1, grad, learning_rate) # Average pool
        grad = model[1].backward(layer0, grad, learning_rate) # Sigmoid
        grad = model[0].backward(x, grad, learning_rate) # CNN with kernel size 3
        
        if i % 100 == 0:
            print("Epoch: " + str(epoch) + ", Iteration: " + str(i) + ", Loss: " + str(loss) + ", Accuracy: " + str(train_correct / 100))
            train_correct = 0
            
    # Test
    test_correct = 0
    for i in range(len(x_test)):
        x = x_test[i]
        y = y_test[i]
        
        # Forward pass
        layer0 = model[0].forward(x) # CNN with kernel size 3
        layer1 = model[1].forward(layer0) # Sigmoid
        layer2 = model[2].forward(layer1) # Average pool
        layer3 = model[3].forward(layer2) # Flatten
        layer4 = model[4].forward(layer3) # Reshape
        layer5 = model[5].forward(layer4) # CNN with kernel size 1
        layer6 = model[6].forward(layer5) # Reshape
        layer7 = model[7].forward(layer6) # Softmax
        
        # accuracy
        test_correct += isCorrect(layer7, y)
    
    print("Epoch: " + str(epoch) + ", Test Accuracy: " + str(test_correct / len(x_test)))

Epoch: 0, Iteration: 0, Loss: 2.588744263314928, Accuracy: 0.0


Epoch: 0, Iteration: 100, Loss: 0.07601143574590724, Accuracy: 0.14
Epoch: 0, Iteration: 200, Loss: 0.5564269202397553, Accuracy: 0.13
Epoch: 0, Iteration: 300, Loss: 0.8250777199999411, Accuracy: 0.15
Epoch: 0, Iteration: 400, Loss: 0.30033168054756704, Accuracy: 0.19
Epoch: 0, Iteration: 500, Loss: 0.2655829394801745, Accuracy: 0.1
Epoch: 0, Iteration: 600, Loss: 0.3069032833647564, Accuracy: 0.18
Epoch: 0, Iteration: 700, Loss: 0.5411470470429335, Accuracy: 0.2
Epoch: 0, Iteration: 800, Loss: 0.12997636776698035, Accuracy: 0.29
Epoch: 0, Iteration: 900, Loss: 0.5426570530837228, Accuracy: 0.1
Epoch: 0, Iteration: 1000, Loss: 0.26815550368887825, Accuracy: 0.18
Epoch: 0, Iteration: 1100, Loss: 0.2648740111711352, Accuracy: 0.18
Epoch: 0, Iteration: 1200, Loss: 0.12339795744686341, Accuracy: 0.27
Epoch: 0, Iteration: 1300, Loss: 0.3772681138534869, Accuracy: 0.22
Epoch: 0, Iteration: 1400, Loss: 0.06410675230751153, Accuracy: 0.2
Epoch: 0, Iteration: 1500, Loss: 0.17511056785547724, A